## Notebook for collecting metadata, processing imaging data through suite2p and analysing trial responses

In [1]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_analysis as ia

QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Plots 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders

Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']


### Run experiments through analysis pipeline using metadata from Google Sheets

In [ ]:
ss_id = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
sheet_names = [
                '2019-06-11_J048',
                '2019-06-13_J052',
                '2019-06-25_RL048',
                '2019-07-15_RL052',
                '2019-08-14_J058',
                '2019-08-14_J059',
                '2019-08-15_RL055',
                '2019-11-14_J061',
                '2019-11-14_J063',
                '2019-11-15_J064',
                '2019-11-18_J065',
                '2020-02-27_RL075',
                '2020-03-03_RL074',
                '2020-03-05_RL070',
                '2020-03-05_RL072',
                '2020-03-12_RL070',
                '2020-03-16_RL074',
                '2020-03-19_RL075',
#                 '2020-03-20_RL080'
              ]

for sheet_name in sheet_names:
    
    # Initialise the class
    exp_obj = ia.interarealProcessing(ss_id, sheet_name, qnap_data_path)
    
#     # Run s2p on this experiment (if not already done)
#     exp_obj.s2pRun()
    
    # Get the range of frames that are attributed to each experiment (from the concatenated t-series)
    exp_obj.getFrameRanges()
    
    # Process the s2p data (after manual curation)
    exp_obj.photostim_r.s2pAnalysis(s2_borders_path)
    exp_obj.photostim_s.s2pAnalysis(s2_borders_path)
    exp_obj.whisker_stim.s2pAnalysis(s2_borders_path)
    
    # Add sham photostim metadata
    exp_obj.addShamPhotostim()
    
    exp_obj.spont.s2pAnalysis(s2_borders_path)
    
    # Pickle the object output to save it for analysis
    pkl_path = os.path.join(pkl_folder, sheet_name + '.pkl')
    
    with open(pkl_path, 'wb') as f:
        pickle.dump(exp_obj, f)


Fetching paths and stim types for: 2019-06-11_J048
Umbrella folder: /home/rlees/mnt/qnap/Data/2019-06-11
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-005']
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_NAPARM/2019-06-11_J048_NAPARM_002']
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-005.paq']
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-006']
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_NAPARM/2019-06-11_J048_NAPARM_006']
['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-006.paq']
Experimental info = (array(['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-005',
       '/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_NAPARM/2019-06-11_J048_NAPARM_002',
       '/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-005.paq'],
      dtype='<U91'), array(['/home/rlees/mnt/qnap/Data/2019-06-11/J048/2019-06-11_J048_t-006',
       '/home/rlees/mnt/qnap/Data/2019-06-11/J048/2